In [ ]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langgraph.store.memory import InMemoryStore
from dotenv import load_dotenv

load_dotenv()

### Identify if question is relevant to be stored in Long-Term-Memory

In [16]:
class GradeQuestion(BaseModel):
    """Boolean value to check whether a question is related to the specified topics."""

    score: str = Field(
        description="Is the question relevant? Respond with 'Yes' or 'No'."
    )

In [17]:
system = """
You are a classifier that examines the given question or statement for any personal information or preferences.
Your job is to determine whether the input contains:
1. Personal information about the user (e.g., name, occupation, location, contact details, or other identifiable information).
2. Preferences, habits, or any explicitly mentioned likes/dislikes.

If you find any such information, respond with 'Yes'. If the input does not contain any personal information or preferences, respond with 'No'.
Your response must be ONLY 'Yes' or 'No'.
"""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Here is the input: {question}"),
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini")
structured_llm = llm.with_structured_output(GradeQuestion)
grader_llm = grade_prompt | structured_llm


In [ ]:
grader_llm.invoke({"question": "Where is Thomas Müller from?"})

In [ ]:
grader_llm.invoke({"question": "Where is Thomas Müller from? I love playing football myself"})

### Summarise question/information

In [25]:
message = "Where is Thomas Müller from? I love playing football myself."

system = """
You are an extractor focused on identifying and summarizing personal information from the given input.
Personal information includes:
1. Names of individuals.
2. Locations.
3. Hobbies, preferences, or habits explicitly mentioned by the user.
4. Any other identifiable personal details.

Your task is to:
- Extract only the personal information present in the input.
- Ignore any irrelevant or general information.
- Provide the extracted personal information as a concise, single-sentence summary.

If no personal information is found, respond with 'No personal information found.'
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", f"Extract and summarize personal information: {message}"),
    ]
)

llm = ChatOpenAI(model="gpt-4o-mini")
summarizer = prompt | llm

In [ ]:
# Invoke the summarizer and print the result
result = summarizer.invoke({})
print(f"Extracted Personal Information: {result}")

In [7]:
from typing import Annotated, Literal, TypedDict, Sequence
from pydantic import BaseModel, Field
from dotenv import load_dotenv

load_dotenv()

from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langgraph.store.memory import InMemoryStore
import uuid
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import MemorySaver


store = InMemoryStore()
USER_ID = "user-123"

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    personal_info_detected: str
    personal_info_extracted: str
    is_duplicate: str
    collected_memories: str

#
# 1) Klassifizierer: Enthält die Nachricht persönliche Infos? (Ja/Nein)
#
class GradeQuestion(BaseModel):
    score: str = Field(description="Yes/No. Does the user's message contain personal info?")

def personal_info_classifier(state: AgentState) -> AgentState:
    message = state["messages"][-1].content
    system_prompt = "You are a classifier. If the message contains personal info, respond 'Yes', otherwise 'No'."
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{message}"),
        ]
    )
    llm = ChatOpenAI(model="gpt-4o-mini")
    structured_llm = llm.with_structured_output(GradeQuestion)
    chain = prompt | structured_llm
    result = chain.invoke({"message": message})
    state["personal_info_detected"] = result.score.strip()
    return state

def personal_info_router(state: AgentState) -> Literal["extract_personal_info","retrieve_memories"]:
    # Falls "Yes", extrahieren. Sonst direkt Memories laden.
    if state["personal_info_detected"].lower() == "yes":
        return "extract_personal_info"
    return "retrieve_memories"

#
# 2) Extrahieren, wenn "Yes"
#
def personal_info_extractor(state: AgentState) -> AgentState:
    message = state["messages"][-1].content
    extractor_system = "You are an extractor. Summarize personal info in one sentence."
    extractor_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", extractor_system),
            ("human", "Input: {message}"),
        ]
    )
    llm = ChatOpenAI(model="gpt-4o-mini")
    chain = extractor_prompt | llm
    extracted_info = chain.invoke({"message": message})
    state["personal_info_extracted"] = extracted_info.strip()
    return state

#
# 3) NEUE Node: Prüft, ob dieses extrahierte Info "neu" ist.
#    Falls neu -> "Yes", sonst -> "No".
#
class InfoNoveltyGrade(BaseModel):
    score: str = Field(description="Yes/No. Is the new info something we have NOT seen yet?")

def personal_info_duplicate_classifier(state: AgentState) -> AgentState:
    new_info = state.get("personal_info_extracted", "")
    namespace = ("memories", USER_ID)
    results = store.search(namespace)
    old_info_list = [doc.value["data"] for doc in results]

    # Prompt an LLM: "Haben wir dieses Info-Fragment bereits? Ja/Nein?"
    system_msg = """
You are a classifier that checks if the new personal info is already stored or not.
If the new info adds anything new, respond 'Yes'. If it is essentially a duplicate, respond 'No'.
"""
    # Wir fassen alte Info in einem String
    old_info_str = "\n".join(old_info_list) if old_info_list else "No stored info so far."

    human_template = """New info:\n{new_info}\n\nExisting memory:\n{old_info}\n
Answer ONLY 'Yes' if the new info is unique. Otherwise 'No'."""
    human_msg = human_template.format(new_info=new_info, old_info=old_info_str)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_msg),
            ("human", "{human_msg}"),
        ]
    )
    llm = ChatOpenAI(model="gpt-4o-mini").with_structured_output(InfoNoveltyGrade)
    chain = prompt | llm
    result = chain.invoke({"human_msg": human_msg})
    state["is_duplicate"] = result.score.strip()
    return state

def personal_info_deduper_router(state: AgentState) -> Literal["personal_info_storer", "retrieve_memories"]:
    # "Yes" => neu => Speichern
    # "No"  => duplikat => skip
    if state["is_duplicate"].lower() == "yes":
        return "personal_info_storer"
    return "retrieve_memories"

#
# 4) Speichern (nur falls NEU)
#
def personal_info_storer(state: AgentState) -> AgentState:
    if state.get("personal_info_extracted"):
        namespace = ("memories", USER_ID)
        store.put(namespace, str(uuid.uuid4()), {"data": state["personal_info_extracted"]})
    return state

#
# 5) Memories abrufen
#
def retrieve_memories(state: AgentState) -> AgentState:
    namespace = ("memories", USER_ID)
    results = store.search(namespace)
    memory_strs = [doc.value["data"] for doc in results]
    state["collected_memories"] = "\n".join(memory_strs)
    return state

#
# 6) Debug-Print
#
def debug_print_personal_memory(state: AgentState) -> AgentState:
    print("=== Personal Memory so far ===")
    if state["collected_memories"]:
        for i, line in enumerate(state["collected_memories"].split("\n"), start=1):
            print(f"Memory {i}: {line}")
    else:
        print("No personal info stored yet.")
    return state

#
# 7) Finaler LLM-Call
#
def call_model(state: AgentState) -> AgentState:
    personal_info = state.get("collected_memories", "")
    system_msg = SystemMessage(
        content=f"You are a helpful assistant. The user has shared these personal details:\n{personal_info}"
    )
    all_messages = [system_msg] + list(state["messages"])
    llm = ChatOpenAI(model="gpt-4o-mini")
    response = llm.invoke(all_messages)
    state["messages"] = state["messages"] + [response]
    return state


#
# GRAPH-DEFINITION
#
workflow = StateGraph(AgentState)

workflow.add_node("personal_info_classifier", personal_info_classifier)
workflow.add_node("personal_info_extractor", personal_info_extractor)
workflow.add_node("personal_info_duplicate_classifier", personal_info_duplicate_classifier)
workflow.add_node("personal_info_storer", personal_info_storer)
workflow.add_node("retrieve_memories", retrieve_memories)
workflow.add_node("debug_print_personal_memory", debug_print_personal_memory)
workflow.add_node("call_model", call_model)

workflow.add_conditional_edges(
    "personal_info_classifier",
    personal_info_router,
    {
        "extract_personal_info": "personal_info_extractor",
        "retrieve_memories": "retrieve_memories",
    },
)

# Nach dem Extrahieren => Duplicate-Check => Speichern ODER skip
workflow.add_edge("personal_info_extractor", "personal_info_duplicate_classifier")
workflow.add_conditional_edges(
    "personal_info_duplicate_classifier",
    personal_info_deduper_router,
    {
        "personal_info_storer": "personal_info_storer",
        "retrieve_memories": "retrieve_memories",
    },
)

workflow.add_edge("personal_info_storer", "retrieve_memories")
workflow.add_edge("retrieve_memories", "debug_print_personal_memory")
workflow.add_edge("debug_print_personal_memory", "call_model")
workflow.add_edge("call_model", END)

workflow.set_entry_point("personal_info_classifier")

checkpointer = MemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import MermaidDrawMethod

display(
    Image(
        graph.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [10]:
input_data_1 = {"messages": [HumanMessage(content="Hi, I'm Thomas Müller and I love playing football. Can you suggest something for dinner?")]}
graph.invoke(input=input_data_1, config={"configurable": {"thread_id": 99}})


=== Personal Memory so far ===
Memory 1: Thomas Müller enjoys playing football.
Memory 2: Thomas Müller enjoys playing football.


{'messages': [HumanMessage(content="Hi, I'm Thomas Müller and I love playing football. Can you suggest something for dinner?", additional_kwargs={}, response_metadata={}, id='ed77e239-5a55-4354-8e71-ebe762763fe8'),
  AIMessage(content='Hi Thomas! Since you enjoy playing football, how about a dinner that’s both delicious and nutritious to fuel your active lifestyle? Here are a few suggestions:\n\n1. **Grilled Chicken with Quinoa Salad**: Grilled chicken breast served over a bed of quinoa mixed with cherry tomatoes, cucumber, bell peppers, and a light vinaigrette.\n\n2. **Pasta Primavera**: Whole grain pasta tossed with fresh vegetables like zucchini, bell peppers, and spinach, drizzled with olive oil and topped with parmesan cheese.\n\n3. **Salmon with Sweet Potato**: Baked salmon served with roasted sweet potatoes and steamed broccoli. It’s packed with protein and healthy fats.\n\n4. **Stir-Fried Tofu and Vegetables**: Tofu stir-fried with a mix of your favorite vegetables, served over

In [ ]:
input_data_2 = {"messages": [HumanMessage(content="What do you know about me?")]}
graph.invoke(input=input_data_2, config={"configurable": {"thread_id": 2}})